In [ ]:
!pip install rich

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 237 kB 5.1 MB/s 
     |████████████████████████████████| 51 kB 3.7 MB/s 


In [ ]:
import json
import os
from rich import box
from rich.console import Console
from rich.prompt import Prompt, Confirm
from rich.panel import Panel
from rich.table import Table

In [ ]:
database = {
    "movies": [
        {
            "name": "Viúva Negra",
            "status": "Alocado"
        },
        {
            "name": "A Lenda dos Guardiões",
            "status": "Alocado"
        },
        {
            "name": "Avatar",
            "status": "Alocado"
        },
        {
            "name": "Eternos",
            "status": "Disponível"
        },
        {
            "name": "Doutor Estranho",
            "status": "Disponível"
        },
        {
            "name": "Deadpool",
            "status": "Disponível"
        },
        {
            "name": "Homem Aranha",
            "status": "Indisponível"
        }
    ]
}

with open("./movies.json", 'w') as file:
    json.dump(database, file, indent=4, ensure_ascii=False)

In [ ]:
from IPython.display import clear_output

In [ ]:
console = Console()

class Blockbuster():
    
    def __init__(self ):
        self.database_movies = self.__load_database()
        self.OPTIONS_METHOD = {
            "1": self.show_movies,
            "2": self.search_movie,
            "3": self.rent_movie,
            "4": self.register_movie,
            "5": self.delete_movie,
            "6": self.update_movie
        }
        self.OPTIONS_STATUS = {
            "1": "Disponível",
            "2": "Alocado",
            "3": "Indisponível"
        }

        self.run()
    
    def __load_database(self, path_to_database='./movies.json'):
        if os.path.exists(path_to_database):
            with open(path_to_database) as file:
                database = json.load(file)
        else:
            database = { "movies": [] }
            self.__save_database(database)
        
        return database

    def __save_database(self, database, path_to_database='./movies.json'):
        with open(path_to_database, 'w') as file:
            json.dump(database, file, indent=4, ensure_ascii=False)

    def run(self):
        option = ""
        while option != "quit":
            console.print(
                Panel(
                    "Bem vindo a Blockbuster",
                    style= "black on white b blink",
                    box= box.HEAVY
                ),
                justify="center"
            )

            console.print(
                """[green u b]Funcionalidades do usuário:[/] 
                1) Mostrar todos os filme do catálogo
                2) Pesquisar por filmes cadastrados no catálogo
                3) Realizar a Locação de um filme do catálogo"""
            )
            
            console.print(
                """[red u b]Funcionalidades de admin:[/] 
                4) Cadastrar um novo filme no catálogo
                5) Deletar um filme do catálogo
                6) Atualizar algum filme do catálogo"""
            )

            option = Prompt.ask("Selecione a funcionalidade ao qual deseja realizar", choices=[str(x) for x in range(1,7)], default="quit")
            
            if option != "quit":
                if option in ["4", "5", "6"]:
                    password = Prompt.ask("Digite a senha de admin")

                    if password == "admin":
                        self.OPTIONS_METHOD[option]()
                    else:
                        console.print("Senha incorreta, finalizando o programa")
                        quit()
                else:
                    self.OPTIONS_METHOD[option]()
                
                option = self.quit_or_continue()

    def show_movies(self):
        t = Table(title='Filmes cadastrados', title_style="b")
        t.add_column('Indíce')
        t.add_column('Nome do Filme')
        t.add_column('Status')
        for index, movie in enumerate(self.database_movies["movies"]):
            t.add_row(str(index + 1), movie["name"],  movie["status"])

        console.print(t, justify="center")

    def search_movie(self):
        word = input("Digite uma palavra ou o nome completo do filme ao qual deseja pesquisar: \n")
        while word == "":
            word = input("Texto vazio, tente novamente: ")

        t = Table(title='Filmes encontrados', title_style="b")
        t.add_column('Indíce')
        t.add_column('Nome do Filme')
        t.add_column('Status')
        
        for index, movie in enumerate(self.database_movies["movies"]):
            if word.lower() in movie["name"].lower():
                t.add_row(str(index + 1), movie["name"],  movie["status"])

        console.print(t, justify="center")

    def rent_movie(self):
        self.show_movies()

        size_movies = len(self.database_movies["movies"])
        index = int(Prompt.ask("Escolha o índice do filme ao qual deseja alugar",  choices= [str(x) for x in range(1,size_movies+1)],  show_choices=False))

        name = self.database_movies["movies"][index-1]["name"]
        status = self.database_movies["movies"][index-1]["status"]
        
        if status != "Alocado":
            rent = Confirm.ask(f"Você confirma a locação do seguinte filme?  Nome: {name} , Status: {status}")

            if rent:
                console.print("O filme foi alocado com sucesso")
                self.database_movies["movies"][index-1]["status"] = "Alocado"
                self.__save_database(self.database_movies)
            else:
                console.print("Nenhum filme foi alocado")
        else:
            console.print("O filme selecionado ja está alocado")

    def register_movie(self):

        name = input("Digite o nome do filme que deseja cadastrar: \n")
        while name == "":
            name = input("Texto vazio, tente novamente: ")
        option_status = Prompt.ask("Digite o número do status do filme a ser cadastrado: (1) Disponível (2) Alocado (3) Indisponível: ", choices=["1", "2", "3"])
        status = self.OPTIONS_STATUS[option_status]
    
        save = Confirm.ask(f"Tem certeza que deseja salvar o seguinte filme?  Nome: {name} , Status: {status}")

        if save:
            self.database_movies["movies"].append({
                "name": name,
                "status": status
            })

            self.__save_database(self.database_movies)

    def update_movie(self):
        self.show_movies()

        size_movies = len(self.database_movies["movies"])
        index = int(Prompt.ask("Escolha o índice do filme ao qual deseja atualizar",  choices= [str(x) for x in range(1,size_movies+1)],  show_choices=False))
        
        name = self.database_movies["movies"][index-1]["name"]
        status = self.database_movies["movies"][index-1]["status"]

        update = Confirm.ask(f"Você tem certeza que deseja atualizar os dados do seguinte filme?  Nome: {name} , Status: {status}")

        if update:
            update_choice = Prompt.ask("Deseja alterar o nome ou o status?",  choices= ['nome', 'status'])

            if update_choice == "nome":

                new_name = input(f"Digite o novo nome do filme {name}: ")
                while new_name == "":
                    new_name = input("Texto vazio, tente novamente: ")
                
                update_name = Confirm.ask(f"Você tem certeza que deseja atualizar o nome do filme {name} para {new_name}?")
                
                if update_name:
                    self.database_movies["movies"][index-1]["name"] = new_name
                    self.__save_database(self.database_movies)
                else:
                    console.print("Nada foi alterado")

            else:
                option_status = Prompt.ask("Digite o número do status do filme a ser alterado: (1) Disponível (2) Alocado (3) Indisponível: ", choices=["1", "2", "3"])
                new_status = self.OPTIONS_STATUS[option_status]
                update_status = Confirm.ask(f"Você tem certeza que deseja atualizar o status do filme {name} de {status} para {new_status}?")

                if update_status:
                    self.database_movies["movies"][index-1]["status"] = new_status
                    self.__save_database(self.database_movies)
                else:
                    console.print("Nada foi alterado")

    def delete_movie(self):
        self.show_movies()

        size_movies = len(self.database_movies["movies"])
        index = int(Prompt.ask("Digite o indice do filme ao qual deseja deletar do banco de dados",  choices= [str(x) for x in range(1,size_movies+1)],  show_choices=False))
        
        name = self.database_movies["movies"][index-1]["name"]
        status = self.database_movies["movies"][index-1]["status"]

        delete = Confirm.ask(f"Tem certeza que deseja deletar o seguinte filme?  Nome: {name} , Status: {status}")

        if delete:
            console.print("O filme foi deletado com sucesso")
            self.database_movies["movies"].pop(index-1)
            self.__save_database(self.database_movies)
        else:
            console.print("Nada foi alterado")

    def quit_or_continue(self):
        choice = Confirm.ask("Deseja continuar ou sair do programa?")
        if choice:
            clear_output()
        else:
            return "quit"

In [ ]:
locadora = Blockbuster()

┏━━━━━━━━━━━━━━━━━━━━━━━━━┓                                            
                                            ┃ Bem vindo a Blockbuster ┃                                            
                                            ┗━━━━━━━━━━━━━━━━━━━━━━━━━┛

Funcionalidades do usuário: 
                1) Mostrar todos os filme do catálogo
                2) Pesquisar por filmes cadastrados no catálogo
                3) Realizar a Locação de um filme do catálogo

Funcionalidades de admin: 
                4) Cadastrar um novo filme no catálogo
                5) Deletar um filme do catálogo
                6) Atualizar algum filme do catálogo

Selecione a funcionalidade ao qual deseja realizar [1/2/3/4/5/6] (quit):

Digite a senha de admin:

               Filmes cadastrados                                                 
                                 ┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓                                 
                                 ┃ Indíce ┃ Nome do Filme         ┃ Status       ┃                                 
                                 ┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩                                 
                                 │ 1      │ Viúva Negra           │ Alocado      │                                 
                                 │ 2      │ A Lenda dos Guardiões │ Alocado      │                                 
                                 │ 3      │ Avatar                │ Alocado      │                                 
                                 │ 4      │ Eternos               │ Disponível   │                                 
                                 │ 5      │ Doutor Estranho       │ Disponível   │                                 
                                 │ 6      │ Deadpool              │ Disponível   │                                 
                                 │ 7      │ Homem Aranha          │ Indisponível │                                 
                                 └────────┴───────────────────────┴──────────────┘

Escolha o índice do filme ao qual deseja atualizar:

In [ ]:
console.print(":status:")

:status: